In [1]:
import datasketch as ds
import networkx as nx

# Load email-Eu-core network dataset
G = nx.read_edgelist('D:\MTech\SEMESTER 2\IT752 Web and Social Computing\WSC Project\Dataset\email-Eu-core\email-Eu-core.txt')

# Step 1: Compute MinHash signatures
num_hashes = 128  # Number of hash functions to use
num_seeds = 10    # Number of seeds to use

# Generate `num_seeds` random seeds
seeds = ds.random_seed(num_seeds)

# Create MinHash objects for each account
minhashes = {}
for account in G.nodes:
    account_set = set(G.neighbors(account))
    mh = ds.MinHash(num_perm=num_hashes, seed=seeds[0])
    for neighbor in account_set:
        mh.update(str(neighbor).encode('utf8'))
    for seed in seeds[1:]:
        mh.merge(ds.MinHash(num_perm=num_hashes, seed=seed))
    minhashes[account] = mh

# Step 2: Compute LSH table
num_bands = 32    # Number of bands to use
band_size = 4     # Number of rows per band

# Create LSH object
lsh = ds.lsh.MinHashLSH(num_perm=num_hashes, num_bands=num_bands, band_size=band_size)

# Insert MinHash signatures into LSH object
for account, mh in minhashes.items():
    lsh.insert(str(account), mh)

# Query LSH object with each seed account
near_accounts = set()
for seed in seeds:
    seed_mh = ds.MinHash(num_perm=num_hashes, seed=seed)
    near_accounts.update(lsh.query(seed_mh))

print(near_accounts)


AttributeError: module 'datasketch' has no attribute 'random_seed'